## Libraries

In [1]:
import torch
import numpy as np
import argparse
import os.path
from sklearn.metrics import roc_auc_score, average_precision_score
from tqdm import tqdm

from utils import prepare_data
from model import LinkPred

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

Device: cpu


In [2]:
# Setting 

def str2bool(v):
    if isinstance(v, bool):
        return v
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')
def str2none(v):
    if v.lower()=='none':
        return None
    else:
        return str(v)
def set_random_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

In [3]:
# Arguments and Parameters

parser = argparse.ArgumentParser(description='Link Prediction with Walk-Pooling')

#Dataset 
parser.add_argument('--data-name', default='cora', help='graph name')

#training/validation/test divison and ratio
parser.add_argument('--use-splitted', type=str2bool, default=True,
                    help='use the pre-splitted train/test data,\
                     if False, then make a random division')
parser.add_argument('--data-split-num',type=str, default='10',
                    help='If use-splitted is true, choose one of splitted data')
parser.add_argument('--test-ratio', type=float, default=0.1,
                    help='ratio of test links')
parser.add_argument('--val-ratio', type=float, default=0.05,
                    help='ratio of validation links. If using the splitted data from SEAL,\
                     it is the ratio on the observed links, othewise, it is the ratio on the whole links.')
parser.add_argument('--practical-neg-sample', type=bool, default = False,
                    help='only see the train positive edges when sampling negative')

#setups in peparing the training set 
parser.add_argument('--observe-val-and-injection', type=str2bool, default = True,
                    help='whether to contain the validation set in the observed graph and apply injection trick')
parser.add_argument('--num-hops', type=int, default=2,
                    help='number of hops in sampling subgraph')
parser.add_argument('--max-nodes-per-hop', type=int, default=None)


#prepare initial node attributes for those graphs do not have
parser.add_argument('--init-attribute', type=str2none, default='ones',
                    help='initial attribute for graphs without node attributes\
                    , options: n2v, one_hot, spc, ones, zeros, None')

#prepare initial node representation using unsupservised models 
parser.add_argument('--init-representation', type=str2none, default= None,
                    help='options: gic, vgae, argva, None')
parser.add_argument('--embedding-dim', type=int, default= 32,
                    help='Dimension of the initial node representation, default: 32)')

#Drnl feature in the SEAL paper
parser.add_argument('--drnl', type=str2bool, default=False,
                    help='whether to use drnl labeling')


#Model and Training
parser.add_argument('--seed', type=int, default=1,
                    help='random seed (default: 1)')
parser.add_argument('--lr', type=float, default=0.00005,
                    help='learning rate')
parser.add_argument('--weight-decay', type=float, default=0)
parser.add_argument('--walk-len', type=int, default=7, help='cutoff in the length of walks')
parser.add_argument('--heads', type=int, default=2,
                    help='using multi-heads in the attention link weight encoder ')
parser.add_argument('--hidden-channels', type=int, default=32)
parser.add_argument('--batch-size', type=int, default=32)
parser.add_argument('--epoch-num', type=int, default=5)
parser.add_argument('--MSE', type=str2bool, default=False)
parser.add_argument('--log', type=str, default=None,
                    help='log by tensorboard, default is None')

args, unknown = parser.parse_known_args()


In [4]:
# Setting argument values based on dataset


#training/validation/test divison and ratio
if  args.data_name in ('cora', 'citeseer','pubmed'):
    args.use_splitted = False
    args.practical_neg_sample = True
    args.observe_val_and_injection = False
    args.init_attribute=None


#setup in peparing the training set     
if (args.data_name in ('Ecoli','PB','pubmed')) and (args.max_nodes_per_hop==None):
    args.max_nodes_per_hop=100
if args.data_name=='Power':
    args.num_hops=3

## Loading dataset

In [5]:

    
print ("-"*45+'Dataset and Features'+"-"*45)
print ("{:<10}|{:<10}|{:<10}|{:<10}|{:<10}|{:<10}|{:<10}|{:<10}|{:<15}|{:<20}"\
    .format('Dataset','Test Ratio','Val Ratio','Split Num','Dimension',\
        'Attribute','hops','DRNL','Representation','Observe val and injection'))
print ("-"*130)
print ("{:<10}|{:<10}|{:<10}|{:<10}|{:<10}|{:<10}|{:<10}|{:<10}|{:<15}|{:<20}"\
    .format(args.data_name,args.test_ratio,args.val_ratio,\
        args.data_split_num,args.embedding_dim,str(args.init_attribute),\
        str(args.num_hops),str(args.drnl),str(args.init_representation),str(args.observe_val_and_injection)))
print ("-"*130)

print('<<Begin generating training data>>')
train_loader, val_loader, test_loader, feature_results = prepare_data(args)
print('<<Complete generating training data>>')

print ("-"*45+'Model and Training'+"-"*45)
print ("{:<13}|{:<13}|{:<13}|{:<8}|{:<13}|{:<8}|{:<15}"\
    .format('Learning Rate','Weight Decay','Batch Size','Epoch',\
        'Walk Length','Heads','Hidden Channels'))
print ("-"*105)

print ("{:<13}|{:<13}|{:<13}|{:<8}|{:<13}|{:<8}|{:<15}"\
    .format(args.lr,args.weight_decay, str(args.batch_size),\
        args.epoch_num,args.walk_len, args.heads, args.hidden_channels))
print ("-"*105)

---------------------------------------------Dataset and Features---------------------------------------------
Dataset   |Test Ratio|Val Ratio |Split Num |Dimension |Attribute |hops      |DRNL      |Representation |Observe val and injection
----------------------------------------------------------------------------------------------------------------------------------
cora      |0.1       |0.05      |10        |32        |None      |2         |False     |None           |False               
----------------------------------------------------------------------------------------------------------------------------------
<<Begin generating training data>>
Using data: cora
Train_link: 8976  Val_link: 526  Test_link: 1054
<<Complete generating training data>>
---------------------------------------------Model and Training---------------------------------------------
Learning Rate|Weight Decay |Batch Size   |Epoch   |Walk Length  |Heads   |Hidden Channels
----------------------------------

## Training

In [7]:
def train(loader,epoch):
    model.train()
    loss_epoch=0
    for data in tqdm(loader,desc="train"):  # Iterate in batches over the training dataset.
        data = data.to(device)
        label= data.label
        out = model(data.x, data.edge_index, data.edge_mask, data.batch, data.z)
        torch.cuda.empty_cache()
        loss = criterion(out.view(-1), label)  
        optimizer.zero_grad()
        loss.backward()  
        optimizer.step()
        loss_epoch=loss_epoch+loss.item()
    return loss_epoch/len(loader)



def test(loader,data_type='test'):
    model.eval()
    scores = torch.tensor([])
    labels = torch.tensor([])
    loss_total=0
    with torch.no_grad():
        #for data in tqdm(loader,position=0,leave=True):  # Iterate in batches over the training/test dataset.
        for data in tqdm(loader,desc='test:'+data_type):  # Iterate in batches over the training/test dataset.
            data = data.to(device)
            out = model(data.x, data.edge_index, data.edge_mask, data.batch, data.z)
            loss = criterion(out.view(-1), data.label)
            out = out.cpu().clone().detach()
            scores = torch.cat((scores,out),dim = 0)
            labels = torch.cat((labels,data.label.view(-1,1).cpu().clone().detach()),dim = 0)
        scores = scores.cpu().clone().detach().numpy()
        labels = labels.cpu().clone().detach().numpy()
        loss_total=loss_total+loss.item()
        return roc_auc_score(labels, scores), average_precision_score(labels, scores),loss_total
       




In [6]:
walk_len = args.walk_len
heads = args.heads
hidden_channels=args.hidden_channels
lr=args.lr
weight_decay=args.weight_decay

torch.cuda.empty_cache()

num_features = next(iter(train_loader)).x.size(1)

z_max=0
if args.drnl==True:
    for data in train_loader:
        z_max = max(z_max, torch.max(data.z).numpy())
    for data in val_loader:
        z_max = max(z_max, torch.max(data.z).numpy())
    for data in test_loader:
        z_max = max(z_max, torch.max(data.z).numpy())
    z_max = z_max+1
    
    #if use drnl, we use a Embedding with dimension = hidden_channels
    num_features = hidden_channels + num_features

torch.cuda.empty_cache()
print("Dimention of features after concatenation:",num_features)
set_random_seed(args.seed)

Dimention of features after concatenation: 1433


## Model

In [6]:
model = LinkPred(in_channels = num_features, hidden_channels = hidden_channels,\
    heads = heads, walk_len = walk_len, drnl = args.drnl,z_max = z_max, MSE= args.MSE).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr,weight_decay=weight_decay)
criterion = torch.nn.MSELoss(reduction='mean')


if args.MSE:
    criterion = torch.nn.MSELoss(reduction='mean')
else:
    criterion = torch.nn.BCEWithLogitsLoss()



In [9]:
Best_Val_fromloss=1e10
Final_Test_AUC_fromloss=0
Final_Test_AP_fromloss=0

Best_Val_fromAUC=0
Final_Test_AUC_fromAUC=0
Final_Test_AP_fromAUC=0

for epoch in range(0, args.epoch_num):
    loss_epoch = train(train_loader,epoch)
    val_auc, val_ap, val_loss = test(val_loader,data_type='val')
    test_auc,test_ap,_ = test(test_loader,data_type='test')
    if val_loss < Best_Val_fromloss:
        Best_Val_fromloss = val_loss
        Final_Test_AUC_fromloss = test_auc
        Final_Test_AP_fromloss = test_ap

    if val_auc > Best_Val_fromAUC:
        Best_Val_fromAUC = val_auc
        Final_Test_AUC_fromAUC = test_auc
        Final_Test_AP_fromAUC = test_ap
    print(f'Epoch: {epoch:03d}, Loss : {loss_epoch:.4f},\
     Val Loss : {val_loss:.4f}, Val AUC: {val_auc:.4f},\
      Test AUC: {test_auc:.4f}, Picked AUC:{Final_Test_AUC_fromAUC:.4f}')


test:test: 100%|███████████████████████████████████████████████████████████████████████| 33/33 [00:14<00:00,  2.27it/s]


Epoch: 000, Loss : 0.4350,     Val Loss : 0.5004, Val AUC: 0.9117,      Test AUC: 0.9220, Picked AUC:0.9220


test:test: 100%|███████████████████████████████████████████████████████████████████████| 33/33 [00:14<00:00,  2.27it/s]


Epoch: 001, Loss : 0.3789,     Val Loss : 0.6731, Val AUC: 0.9151,      Test AUC: 0.9262, Picked AUC:0.9262


test:test: 100%|███████████████████████████████████████████████████████████████████████| 33/33 [00:15<00:00,  2.18it/s]


Epoch: 002, Loss : 0.3551,     Val Loss : 0.3204, Val AUC: 0.9185,      Test AUC: 0.9317, Picked AUC:0.9317


test:test: 100%|███████████████████████████████████████████████████████████████████████| 33/33 [00:14<00:00,  2.20it/s]


Epoch: 003, Loss : 0.3411,     Val Loss : 0.2588, Val AUC: 0.9233,      Test AUC: 0.9270, Picked AUC:0.9270


test:test: 100%|███████████████████████████████████████████████████████████████████████| 33/33 [00:28<00:00,  1.14it/s]

Epoch: 004, Loss : 0.3331,     Val Loss : 0.4443, Val AUC: 0.9175,      Test AUC: 0.9257, Picked AUC:0.9270


In [11]:
test_auc,test_ap,_ = test(test_loader,data_type='test')

test:test: 100%|███████████████████████████████████████████████████████████████████████| 33/33 [00:26<00:00,  1.22it/s]


In [12]:
print(test_auc)

0.9221399277713166


In [13]:
print(f'From loss: Final Test AUC: {Final_Test_AUC_fromloss:.4f}, Final Test AP: {Final_Test_AP_fromloss:.4f}')
print(f'From AUC: Final Test AUC: {Final_Test_AUC_fromAUC:.4f}, Final Test AP: {Final_Test_AP_fromAUC:.4f}')

From loss: Final Test AUC: 0.0000, Final Test AP: 0.0000
From AUC: Final Test AUC: 0.0000, Final Test AP: 0.0000


##  Measurement

In [8]:
import statistics as stat
import time


Eva_final=dict()
Base_model_Auc=[]
T_base_model=[]

In [9]:
for i in range(5):
    Best_Val_fromloss=1e10
    Final_Test_AUC_fromloss=0
    Final_Test_AP_fromloss=0

    Best_Val_fromAUC=0
    Final_Test_AUC_fromAUC=0
    Final_Test_AP_fromAUC=0

    for epoch in range(0, args.epoch_num):
        loss_epoch = train(train_loader,epoch)
        val_auc, val_ap, val_loss = test(val_loader,data_type='val')
        test_auc,test_ap,_ = test(test_loader,data_type='test')
        if val_loss < Best_Val_fromloss:
            Best_Val_fromloss = val_loss
            Final_Test_AUC_fromloss = test_auc
            Final_Test_AP_fromloss = test_ap

        if val_auc > Best_Val_fromAUC:
            Best_Val_fromAUC = val_auc
            Final_Test_AUC_fromAUC = test_auc
            Final_Test_AP_fromAUC = test_ap
        print(f'Epoch: {epoch:03d}, Loss : {loss_epoch:.4f},\
         Val Loss : {val_loss:.4f}, Val AUC: {val_auc:.4f},\
          Test AUC: {test_auc:.4f}, Picked AUC:{Final_Test_AUC_fromAUC:.4f}')
        
        
        
    print(f'From loss: Final Test AUC: {Final_Test_AUC_fromloss:.4f}, Final Test AP: {Final_Test_AP_fromloss:.4f}')
    print(f'From AUC: Final Test AUC: {Final_Test_AUC_fromAUC:.4f}, Final Test AP: {Final_Test_AP_fromAUC:.4f}')
        
        
    start = time.time()
    test_auc,test_ap,_ = test(test_loader,data_type='test')
    end = time.time()
    t_inference= end-start
        
    Base_model_Auc.append(test_auc)
    T_base_model.append(t_inference)
    print(f"Time inference: {t_inference},Test Auc: {test_auc:.3f},")

    

test:test: 100%|███████████████████████████████████████████████████████████████████████| 33/33 [00:29<00:00,  1.12it/s]


Epoch: 000, Loss : 0.4350,         Val Loss : 0.5004, Val AUC: 0.9117,          Test AUC: 0.9220, Picked AUC:0.9220


test:test: 100%|███████████████████████████████████████████████████████████████████████| 33/33 [00:23<00:00,  1.39it/s]


Epoch: 001, Loss : 0.3789,         Val Loss : 0.6731, Val AUC: 0.9151,          Test AUC: 0.9262, Picked AUC:0.9262


test:test: 100%|███████████████████████████████████████████████████████████████████████| 33/33 [00:38<00:00,  1.17s/it]


...

Epoch: 003, Loss : 0.1659,         Val Loss : 0.2000, Val AUC: 0.9162,          Test AUC: 0.9152, Picked AUC:0.9187


test:test: 100%|███████████████████████████████████████████████████████████████████████| 33/33 [00:41<00:00,  1.25s/it]


Epoch: 004, Loss : 0.1622,         Val Loss : 0.3194, Val AUC: 0.9206,          Test AUC: 0.9175, Picked AUC:0.9187
From loss: Final Test AUC: 0.9213, Final Test AP: 0.9344
From AUC: Final Test AUC: 0.9187, Final Test AP: 0.9326


test:test: 100%|███████████████████████████████████████████████████████████████████████| 33/33 [00:13<00:00,  2.37it/s]

Time inference: 13.95359206199646,Test Auc: 0.918,


In [10]:
Base_model_Auc

[0.9257117549841752,
 0.9180604114082433,
 0.9178623766333369,
 0.9170018255205614,
 0.9175023134062341]

In [11]:
T_base_model

[14.00715160369873,
 13.769235372543335,
 14.05490231513977,
 13.637200593948364,
 13.95359206199646]

In [13]:
print(f'Base_model_Auc in ten time:')
print ("{:<10}|{:<10}|{:<10}|{:<10}|{:<10}"\
    .format(Base_model_Auc[0],Base_model_Auc[1],Base_model_Auc[2],\
        Base_model_Auc[3],Base_model_Auc[4]))
print ("-"*110)
print(f'Time Inference model in ten time:')
#print(T_base_model)
print ("{:<10}|{:<10}|{:<10}|{:<10}|{:<10}"\
    .format(T_base_model[0],T_base_model[1],T_base_model[2],\
        T_base_model[3],T_base_model[4]))

Base_model_Auc in ten time:
0.9257117549841752|0.9180604114082433|0.9178623766333369|0.9170018255205614|0.9175023134062341
--------------------------------------------------------------------------------------------------------------
Time Inference model in ten time:
14.00715160369873|13.769235372543335|14.05490231513977|13.637200593948364|13.95359206199646


In [14]:
import statistics as stat

base_model_accuracy_mean = stat.mean(Base_model_Auc)
base_model_accuracy_std =  stat.stdev(Base_model_Auc)
desc_auc = "{:.3f} ± {:.3f}".format(base_model_accuracy_mean,base_model_accuracy_std)
print(f"Auc: {desc_auc}"  )

Eva_final.update({'base model accuracy':float(format(base_model_accuracy_mean, '.4f'))})
                 
t_base_model_mean =stat.mean(T_base_model)
t_base_model_std =stat.stdev(T_base_model)   
desc_T = "{:.3f} ± {:.3f}".format(t_base_model_mean,t_base_model_std)

print(f"Time inference :{desc_T}")
Eva_final.update({'time inference of base model':float(format(t_base_model_mean, '.6f'))})

Auc: 0.919 ± 0.004
Time inference :13.884 ± 0.176


In [15]:
Eva_final

{'base model accuracy': 0.9192, 'time inference of base model': 13.884416}